In [48]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timezone
from scipy.optimize import minimize, least_squares
from scipy.integrate import quad

# --- Heston Pricing Functions ---
def heston_characteristic_function(phi, S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2):
    x = np.log(S)
    a = kappa * theta
    u = 0.5 if P1P2 == 1 else -0.5
    b = kappa - rho * volvol if P1P2 == 1 else kappa
    d = np.sqrt((rho * volvol * phi * 1j - b)**2 - volvol**2 * (2 * u * phi * 1j - phi**2))
    g = (b - rho * volvol * phi * 1j + d) / (b - rho * volvol * phi * 1j - d)
    C = (r - div) * phi * 1j * T + (a / volvol**2) * ((b - rho * volvol * phi * 1j + d) * T - 2 * np.log((1 - g * np.exp(d * T)) / (1 - g)))
    D = (b - rho * volvol * phi * 1j + d) / volvol**2 * (1 - np.exp(d * T)) / (1 - g * np.exp(d * T))
    return np.exp(C + D * var0 + 1j * phi * x)

def heston_call_price(S, K, T, r, kappa, rho, volvol, theta, var0, div):
    def integrand(phi, P1P2):
        cf = heston_characteristic_function(phi, S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2)
        return np.real(np.exp(-1j * phi * np.log(K)) * cf / (1j * phi))

    eps = 1e-6           # lower integration limit (avoids φ = 0)
    integral_P1 = quad(lambda phi: integrand(phi, 1),
                    eps, 100, limit=200, epsabs=1e-8, epsrel=1e-8)[0]
    integral_P2 = quad(lambda phi: integrand(phi, 2),
                    eps, 100, limit=200, epsabs=1e-8, epsrel=1e-8)[0]
    
    P1 = 0.5 + (1 / np.pi) * integral_P1
    P2 = 0.5 + (1 / np.pi) * integral_P2
    return max(0.0, S * np.exp(-div * T) * P1 - K * np.exp(-r * T) * P2)

def heston_put_price(S, K, T, r, kappa, rho, volvol, theta, var0, div):
    """Put option pricing function using put-call parity"""
    CallValue = heston_call_price(S, K, T, r, kappa, rho, volvol, theta, var0, div)
    PutValue = CallValue - S * np.exp(-div * T) + K * np.exp(-r * T)
    return PutValue

def heston_prices_parallel(params, Spots, Strikes, Maturities, Rates, div):
    kappa, rho, volvol, theta, var0 = params
    return np.array([
        heston_call_price(S, K, T, r, kappa, rho, volvol, theta, var0, div)
        for S, K, T, r in zip(Spots, Strikes, Maturities, Rates)
    ])


def OptFunctionFast(params, Spots, Maturities, Rates, Strikes, MarketP, div, check_bounds=True):
    kappa, rho, volvol, theta, var0 = params
    if check_bounds and not (0.1 <= kappa <= 15 and -0.99 <= rho <= 0 and 0.01 <= volvol <= 2 and 0.001 <= theta <= 0.5 and 0.001 <= var0 <= 0.5):
        return 1e10

    mask = np.isfinite(MarketP) & (MarketP > 0)
    if not np.any(mask):
        return 1e10

    S, T, r, K, Pmkt = Spots[mask], Maturities[mask], Rates[mask], Strikes[mask], MarketP[mask]
    Pmodel = heston_prices_parallel(params, S, K, T, r, div)
    
    # Plain Squared Erro
    err = np.mean((Pmodel - Pmkt)**2)
    
    # Log-Moneyness Weighted Error
    # log_m = np.log(K / S)
    # weights = 1.0 + np.exp(-10 * log_m**2)
    # err = np.mean((Pmodel - Pmkt)**2 * weights)
    
    return err if np.isfinite(err) else 1e10


In [ ]:
import yfinance as yf
from math import exp
import requests

FRED_API_KEY = ""
FRED_SERIES = {
    '1M': 'DGS1MO', '3M': 'DGS3MO', '6M': 'DGS6MO',
    '1Y': 'DGS1', '2Y': 'DGS2'
}

def fetch_fred_rates():
    rates = {}
    for label, sid in FRED_SERIES.items():
        url = f"https://api.stlouisfed.org/fred/series/observations?series_id={sid}&api_key=0de8b88e8310c6ebbd66c2eaa2ccb03f&file_type=json&sort_order=desc&limit=1"
        try:
            val = requests.get(url).json()['observations'][0]['value']
            if val != '.':
                rates[label] = float(val) / 100
        except:
            continue
    return rates

def get_rate_key(T):
    return (
        '1M' if T <= 1/12 else
        '3M' if T <= 0.25 else
        '6M' if T <= 0.5 else
        '1Y' if T <= 1 else
        '2Y'
    )

def get_option_calibration_data(symbol, target_expiration_str, max_main=20, max_side=15):
    tk = yf.Ticker(symbol)
    expirations = pd.to_datetime(tk.options).date
    target_exp = pd.to_datetime(target_expiration_str).date()
    if target_exp not in expirations:
        raise ValueError("Target expiration not available.")
    
    idx = np.where(expirations == target_exp)[0][0]
    selected_dates = [expirations[i] for i in range(idx - 10, idx + 10) if 0 <= i < len(expirations)]

    spot = tk.history(period="1d")['Close'].iloc[-1]
    rates = fetch_fred_rates()
    today = datetime.today().date()
    data = []

    for expiry in selected_dates:
        try:
            df = tk.option_chain(expiry.isoformat()).calls
            df = df.dropna(subset=["bid", "ask", "impliedVolatility", "volume", "openInterest"])
            df = df[(df.bid > 0) & (df.ask > 0)].copy()
            df["midPrice"] = (df.bid + df.ask) / 2

            T = (expiry - today).days / 365
            rate = rates.get(get_rate_key(T), 0.0)
            F = spot * exp(rate * T)

            atm_strike = df.loc[(df.strike - F).abs().idxmin(), "strike"]
            df["moneyness"] = (df.strike - atm_strike).abs()
            n = max_main if expiry == target_exp else max_side

            selected = df.nsmallest(n, "moneyness").copy()
            selected["maturityDate"] = expiry
            selected["maturity"] = T
            selected["rate"] = rate
            selected["forward"] = F
            data.append(selected[[
                "maturityDate", "maturity", "strike", "midPrice",
                "impliedVolatility", "forward", "rate"
            ]])
        except:
            continue

    return pd.concat(data).reset_index(drop=True) if data else pd.DataFrame()

In [62]:
def Feller(x):
    kappa, rho, volvol, theta, var0 = x
    return 2 * kappa * theta - volvol**2

def calibrate_heston(symbol, expiration, div=0.0):
    t0 = time.time()
    data = get_option_calibration_data(symbol, expiration, max_main=30, max_side=20)
    
    if data.empty:
        return {"success": False, "error": "No data found"}

    print(f"Data loaded in {time.time() - t0:.2f} seconds | {len(data)} options used")

    Spots      = data.forward.values
    Strikes    = data.strike.values
    Maturities = data.maturity.values
    Rates      = data.rate.values
    MarketP    = data.midPrice.values

    avg_iv = np.mean(data.impliedVolatility)
    var0 = avg_iv**2
    init = [1.5, -0.7, 0.6 * avg_iv, var0, var0]
    bounds = [(0.1, 10), (-0.95, 0.0), (0.01, 1.5), (0.001, 0.4), (0.001, 0.4)]
    cons = {'type': 'ineq', 'fun': Feller}

    t1 = time.time()
    result = minimize(
        OptFunctionFast, init, args=(Spots, Maturities, Rates, Strikes, MarketP, div, True),
        method="SLSQP", bounds=bounds, constraints=cons, options={"maxiter": 500, "disp": False}
    )
    # result = minimize(OptFunctionFast, init, args=(Spots, Maturities, Rates, Strikes, MarketP, div, True), 
    #               method='nelder-mead', options={'xtol': 1e-8, 'disp': True})
    elapsed = time.time() - t1

    xopt = result.x
    modelP = heston_prices_parallel(xopt, Spots, Strikes, Maturities, Rates, div)
    mse = np.mean((modelP - MarketP) ** 2)

    print(result)
    print(f"Calibration time: {elapsed:.2f}s | MSE: {mse:.6f}")
    print(f"Feller condition: {Feller(xopt):.8f} > 0")
    print(f"Parameters: kappa={xopt[0]:.4f}, rho={xopt[1]:.4f}, volvol={xopt[2]:.4f}, theta={xopt[3]:.4f}, var0={xopt[4]:.4f}")

    return {
        "result": result,
        "success": result.success,
        "mse": mse,
        "parameters": dict(zip(["kappa", "rho", "volvol", "theta", "var0"], xopt)),
        "optimization_time": elapsed,
        "total_time": time.time() - t0
    }

In [61]:
res = calibrate_heston(
    symbol='SPY',
    expiration='2025-07-31', 
)

Data loaded in 9.78 seconds | 305 options used
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.7293214871952328
       x: [ 3.997e+00 -9.324e-01  1.689e-01  5.733e-03  5.266e-02]
     nit: 18
     jac: [ 5.966e-02  1.214e+00 -2.263e-01  2.318e+01  8.225e+00]
    nfev: 138
    njev: 18
Calibration time: 329.60s | MSE: 0.729321
Feller condition: 0.01729772 > 0


In [ ]:
import pandas as pd

# --- Calibration Function ---
def calibrate_heston(symbol, expiration, s0,
                                      div, max_options, max_time=300,
                                      methods=None,
                                      save_excel_path=None):
    if methods is None:
        methods = ["L-BFGS-B", "SLSQP", "LM"]

    start = time.time()
    
    data = get_clean_market_data(symbol, expiration, s0, max_options, min_maturity_days=5)
    
    print(f"Data loaded in {time.time() - start:.2f} seconds")
    print(f"Market data used: {len(data)} options")
    
    if data.empty:
        return {"success": False, "error": "No data found"}

    Spots    = data.forward.values
    Strikes  = data.strike.values
    Maturities = data.T.values
    Rates    = data.rate.values
    MarketP  = data.midPrice.values

    avg_iv   = np.mean(data.impliedVolatility)
    init_var = avg_iv ** 2
    initial_guess = [1.5, -0.7, 0.3 * avg_iv, init_var, init_var]
    bounds = [(0.1, 10.0), (-0.95, 0.0), (0.01, 1.5),
              (0.001, 0.4), (0.001, 0.4)]

    results = []
    detailed_results = []

    for method in methods:
        if time.time() - start > max_time:
            break
        try:
            t0 = time.time()
            if method == "LM":
                def residuals(p, Spots, Strikes, Mats, Rates, Market, div):
                    kappa, rho, volvol, theta, var0 = p
                    if not (0.1 <= kappa <= 15.0 and -0.99 <= rho <= 0.0
                            and 0.01 <= volvol <= 2.0 and 0.001 <= theta <= 0.5
                            and 0.001 <= var0 <= 0.5):
                        return np.full_like(Market, 1e5)
                    model = heston_prices_parallel(p, Spots, Strikes, Mats, Rates, div)
                    return model - Market

                res = least_squares(
                    residuals, initial_guess, method='lm',
                    args=(Spots, Strikes, Maturities, Rates, MarketP, div),
                    xtol=1e-12, ftol=1e-12, gtol=1e-12)
                xopt = res.x
                model_prices = heston_prices_parallel(
                    xopt, Spots, Strikes, Maturities, Rates, div)

            else:
                res = minimize(
                    OptFunctionFast, initial_guess, method=method, bounds=bounds,
                    args=(Spots, Maturities, Rates, Strikes, MarketP, div, True),
                    options={"maxiter": 500, "disp": False})
                xopt = res.x
                model_prices = heston_prices_parallel(
                    xopt, Spots, Strikes, Maturities, Rates, div)

            elapsed = time.time() - t0
            total_mse = np.mean((model_prices - MarketP) ** 2)
            per_option_mse = (model_prices - MarketP) ** 2

            # save full details
            detailed_results.append({
                "method": method,
                "total_mse": total_mse,
                "time": elapsed,
                "params": xopt,
                "model_prices": model_prices,
                "market_prices": MarketP,
                "per_option_mse": per_option_mse,
                "strikes": Strikes,
                "maturities": Maturities
            })

            results.append((method, total_mse, xopt, elapsed))
            print(f"Method: {method:11s} | Total MSE: {total_mse:.6f} | Time: {elapsed:.2f} s")

        except Exception as err:
            print(f"Method: {method:11s} failed → {err}")
            continue

    if not results:
        return {"success": False, "error": "All methods failed"}

    best_method, best_mse, best_params, best_time = min(results, key=lambda t: t[1])

    # --------- Save to Excel --------------
    if save_excel_path:
        writer = pd.ExcelWriter(save_excel_path, engine='xlsxwriter')

        for detail in detailed_results:
            df = pd.DataFrame({
                "Strike": detail["strikes"],
                "Maturity (years)": detail["maturities"],
                "Market Price": detail["market_prices"],
                "Model Price": detail["model_prices"],
                "Per Option MSE": detail["per_option_mse"],
                "Absolute Error": np.abs(detail["model_prices"] - detail["market_prices"]),
                "Relative Error (%)": 100 * np.abs(detail["model_prices"] - detail["market_prices"]) / detail["market_prices"]
            })

            sheet_name = detail["method"][:30]  # Excel max 31 chars
            df.to_excel(writer, sheet_name=sheet_name, index=False)

        # summary sheet
        summary = pd.DataFrame([{
            "Method": d["method"],
            "Total MSE": d["total_mse"],
            "Time (s)": d["time"],
            "Kappa": d["params"][0],
            "Rho": d["params"][1],
            "VolVol": d["params"][2],
            "Theta": d["params"][3],
            "V0": d["params"][4]
        } for d in detailed_results])
        summary.to_excel(writer, sheet_name="Summary", index=False)

        writer.close()  # Change from writer.save() to writer.close()
        print(f"Results saved to {save_excel_path}")

    # --------- Done --------------
    return {
        "success": True,
        "best_method": best_method,
        "mse": best_mse,
        "parameters": dict(zip(["kappa", "rho", "volvol", "theta", "var0"],
                               best_params)),
        "market_data_used": len(data),
        "calibration_time": time.time() - start,
        "all_results": [
            {"method": m, "mse": e, "params": p.tolist(), "time": t}
            for m, e, p, t in results
        ]
    }
